# Haiku aus dem Nationalrat

Der korrekte Plural von Haiku ist Haiku.  
Der Verständlichkeit halber wird im Code aber *Haikus* verwendet.

protokolle laden und flatten

In [1]:
import json
import pandas as pd

with open("woswormeileistung/data/sessions.json") as f:
    sessions = json.load(f)

wortmeldungen = pd.json_normalize(
    sessions,
    record_path=["sections"],
    meta=["period", "sessionNumber", "date"],
)

wortmeldungen = wortmeldungen[["period", "sessionNumber", "date", "speaker", "text"]]
wortmeldungen = wortmeldungen.rename(columns={"sessionNumber": "session"})

wortmeldungen.head()

,period,session,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
1,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
2,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Der Herr Bund...
3,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Die Amtlichen...
4,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Ich darf beka...


sprecher:in label aus text entfernen

In [2]:
wortmeldungen["text"] = wortmeldungen["text"].str.split(": ", n=1).str[1]
wortmeldungen.head()

,period,session,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren Abgeordne...
1,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren auf der G...
2,XXVII,276,2024-09-18T00:00:00,88386,Der Herr Bundespräsident hat mit Entschließung...
3,XXVII,276,2024-09-18T00:00:00,88386,Die Amtlichen Protokolle der 272. und der 273....
4,XXVII,276,2024-09-18T00:00:00,88386,"Ich darf bekannt geben, dass von der Bundeswah..."


dev mode: probelauf auf 1000 texte

In [ ]:
#wortmeldungen = wortmeldungen.sample(1000)

haiku finden

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex
import pyphen
import regex
from tqdm import tqdm

!python -m spacy download de_core_news_lg
#!python -m spacy download de_core_news_sm

nlp = spacy.load("de_core_news_lg")
#nlp = spacy.load("de_core_news_sm")

# bindestrich-wörter nicht trennen
infixes = [x for x in nlp.Defaults.infixes if "-" not in x and "–" not in x and "—" not in x]
infix_re = compile_infix_regex(infixes)
nlp.tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=nlp.tokenizer.prefix_search,
    suffix_search=nlp.tokenizer.suffix_search,
    infix_finditer=infix_re.finditer,
)

dic = pyphen.Pyphen(lang="de_DE")


def count_syllables(word: str) -> int:
    parts = word.split("-")
    total = 0
    for part in parts:
        if not part:
            continue
        hyphenated = dic.inserted(part, hyphen="·")
        total += hyphenated.count("·") + 1
    return total


def extract_haiku(sentence):
    pattern = r"^[\p{L} .!?'\":;,–—―]+$"
    if not bool(regex.fullmatch(pattern, sentence)):
        return None
    
    common_phrases = ["zu wort gemeldet", "nächste rednerin", "nächster redner", "damen und herren"]
    if any(phrase in sentence.lower() for phrase in common_phrases):
        return None
    
    tokens = nlp(sentence)
    line_limits = [5, 7, 5]
    line_idx = 0
    line_sum = 0
    extracted_haiku = [[], [], []]

    for i in range(len(tokens)):
        token = tokens[i]
        if token.text in [",", ":", ";", "–", "—", "―"]:
            if line_sum == 0:
                continue
            return None

        if token.is_punct:
            continue

        if line_sum == 0 and token.head.i < i:
            return None

        extracted_haiku[line_idx].append(token.text)
        line_sum += count_syllables(token.text)
        if line_sum > line_limits[line_idx]:
            return None
        
        if line_sum == line_limits[line_idx]:
            if token.head.i > i:
                return None
            
            line_idx += 1
            line_sum = 0
            if line_idx == 3:
                remaining_syllables = sum([count_syllables(t.text) for t in tokens[i + 1 :] if not t.is_punct])
                if remaining_syllables > 0:
                    return None
                break

    return [" ".join(line) for line in extracted_haiku] if line_idx == 3 else None


haikus = []
context_indices = []
for doc in tqdm(nlp.pipe(wortmeldungen["text"].fillna("").astype(str), batch_size=50), total=len(wortmeldungen)):
    haikus.append([extract_haiku(sentence.text) for sentence in doc.sents])
    context_indices.append([(sentence.start_char, sentence.end_char) for sentence in doc.sents])
    
wortmeldungen["lines"] = haikus
wortmeldungen["context_indices"] = context_indices
wortmeldungen.head()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.8.0/de_core_news_sm-3.8.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


100%|██████████| 1000/1000 [01:28<00:00, 11.28it/s]


,period,session,date,speaker,text,lines,context_indices
120965,XXIV,77,2010-09-22T00:00:00,51586,Herr Präsident! Herr Landwirtschaftsminister! ...,"[None, None, None, None, None, None, None, Non...","[(0, 15), (16, 45), (46, 84), (85, 139), (139,..."
89650,XXV,14,2014-02-25T00:00:00,47087,Herr Präsident! Herr Bundesminister! Liebe Kol...,"[None, None, None, None, None, None, None, Non...","[(0, 15), (16, 36), (37, 68), (69, 160), (161,..."
62604,XXV,165,2017-03-01T00:00:00,2822,Als Nächste zu Wort gelangt Frau Abgeordnete S...,"[None, None, None]","[(0, 52), (53, 62), (62, 70)]"
73044,XXV,104,2015-11-24T00:00:00,83143,Herr Präsident! Herr Minister! Hohes Haus! Wen...,"[None, None, None, None, None, None, None, Non...","[(0, 15), (16, 30), (31, 42), (43, 266), (266,..."
10953,XXVII,213,2023-05-24T00:00:00,18666,Frau Präsidentin! Frau Präsidentin des Rechnun...,"[None, None, None, None, None, None, None, Non...","[(0, 17), (18, 54), (55, 80), (81, 112), (113,..."


liste auf haikus flatten und contextBefore/after ableiten

In [5]:
haikus = wortmeldungen.explode(["lines", "context_indices"]).dropna(subset=["lines"])
haikus["line1"] = haikus["lines"].str[0]
haikus["line2"] = haikus["lines"].str[1]
haikus["line3"] = haikus["lines"].str[2]
haikus["context_before"] = haikus.apply(lambda row: row["text"][: row["context_indices"][0]], axis=1)
haikus["context_after"] = haikus.apply(lambda row: row["text"][row["context_indices"][1] :], axis=1)
haikus = haikus.drop(columns=["text", "context_indices", "lines"])
haikus.head()

,period,session,date,speaker,line1,line2,line3,context_before,context_after
81126,XXV,59,2015-01-21T00:00:00,51561,Es wird jedenfalls,gut sein für Österreich und,die Österreicher,Sehr geehrte Damen und Herren! Sehr geehrte Zu...,"Das meint offenbar auch die ÖVP, weil sie sag..."
93464,XXIV,213,2013-07-03T00:00:00,30011,Diesbezüglich hat,die Zukunft im Burgenland,bereits begonnen,Herr Präsident! Herr Minister! Kolleginnen und...,Sämtliche Gemeinden rund um den Neusiedler Se...
126631,XXIV,51,2009-12-11T00:00:00,15465,Ich verstehe schon,dass Sie nicht hören wollen,wie die Zahlen sind,Sehr geehrte Frau Präsidentin! Hohes Haus! Mei...,Sie liegen aber auf dem Tisch. (Abg. Dr. Glaw...
81810,XXV,55,2014-12-11T00:00:00,83139,Interessant ist,auch die Frage zur Zukunft,des ländlichen Raums,Sehr geehrter Herr Präsident! Sehr geehrter He...,"Wir wollen nämlich, dass auch die auf dem Lan..."
17332,XXVII,171,2022-09-21T00:00:00,78586,Warum müssen wir,das schon wieder so machen,wie bei der Cofag,Der Beginn dieser kurzen Debatte ist ein Sinnb...,Warum müssen wir schon wieder eine Blackbox m...


gesetzgebungsperiode als arabische zahl für tooltip

In [6]:
import roman

haikus["period_roman"] = haikus["period"]
haikus["period"] = haikus["period"].apply(lambda periodRoman: roman.fromRoman(periodRoman))
haikus.head()

,period,session,date,speaker,line1,line2,line3,context_before,context_after,period_roman
81126,25,59,2015-01-21T00:00:00,51561,Es wird jedenfalls,gut sein für Österreich und,die Österreicher,Sehr geehrte Damen und Herren! Sehr geehrte Zu...,"Das meint offenbar auch die ÖVP, weil sie sag...",XXV
93464,24,213,2013-07-03T00:00:00,30011,Diesbezüglich hat,die Zukunft im Burgenland,bereits begonnen,Herr Präsident! Herr Minister! Kolleginnen und...,Sämtliche Gemeinden rund um den Neusiedler Se...,XXIV
126631,24,51,2009-12-11T00:00:00,15465,Ich verstehe schon,dass Sie nicht hören wollen,wie die Zahlen sind,Sehr geehrte Frau Präsidentin! Hohes Haus! Mei...,Sie liegen aber auf dem Tisch. (Abg. Dr. Glaw...,XXIV
81810,25,55,2014-12-11T00:00:00,83139,Interessant ist,auch die Frage zur Zukunft,des ländlichen Raums,Sehr geehrter Herr Präsident! Sehr geehrter He...,"Wir wollen nämlich, dass auch die auf dem Lan...",XXV
17332,27,171,2022-09-21T00:00:00,78586,Warum müssen wir,das schon wieder so machen,wie bei der Cofag,Der Beginn dieser kurzen Debatte ist ein Sinnb...,Warum müssen wir schon wieder eine Blackbox m...,XXVII


personen verknüpfen

In [7]:
personen = pd.read_json("woswormeileistung/data/persons.json")
personen = personen[["id", "name", "parties", "imageUrl"]]
personen = personen.rename(columns={"imageUrl": "image_url"})
personen["id"] = personen["id"].astype(str)
haikus = haikus.merge(personen, left_on="speaker", right_on="id", how="left")
haikus = haikus.drop(columns=["id"])
haikus = haikus.rename(columns={"name": "person_name", "speaker": "person_id"})
haikus["parties"] = haikus["parties"].apply(
    lambda x: x if isinstance(x, list) and len(x) > 0 else ["Ohne Klub"]
)
haikus.head()

,period,session,date,person_id,line1,line2,line3,context_before,context_after,period_roman,person_name,parties,image_url
0,25,59,2015-01-21T00:00:00,51561,Es wird jedenfalls,gut sein für Österreich und,die Österreicher,Sehr geehrte Damen und Herren! Sehr geehrte Zu...,"Das meint offenbar auch die ÖVP, weil sie sag...",XXV,Dr. Johannes Hübner,[FPÖ],https://parlament.gv.at/dokument/bild/200695/2...
1,24,213,2013-07-03T00:00:00,30011,Diesbezüglich hat,die Zukunft im Burgenland,bereits begonnen,Herr Präsident! Herr Minister! Kolleginnen und...,Sämtliche Gemeinden rund um den Neusiedler Se...,XXIV,Erwin Preiner,[SPÖ],https://parlament.gv.at/dokument/bild/72616/72...
2,24,51,2009-12-11T00:00:00,15465,Ich verstehe schon,dass Sie nicht hören wollen,wie die Zahlen sind,Sehr geehrte Frau Präsidentin! Hohes Haus! Mei...,Sie liegen aber auf dem Tisch. (Abg. Dr. Glaw...,XXIV,Dipl.-Ing. Josef Pröll,[ÖVP],https://parlament.gv.at/dokument/bild/21016/21...
3,25,55,2014-12-11T00:00:00,83139,Interessant ist,auch die Frage zur Zukunft,des ländlichen Raums,Sehr geehrter Herr Präsident! Sehr geehrter He...,"Wir wollen nämlich, dass auch die auf dem Lan...",XXV,Dr. Kathrin Nachbaur,"[STRONACH, ÖVP]",https://parlament.gv.at/dokument/bild/64016/64...
4,27,171,2022-09-21T00:00:00,78586,Warum müssen wir,das schon wieder so machen,wie bei der Cofag,Der Beginn dieser kurzen Debatte ist ein Sinnb...,Warum müssen wir schon wieder eine Blackbox m...,XXVII,"Christian Hafenecker, MA",[FPÖ],https://parlament.gv.at/dokument/bild/201431/2...


möglichst stabile id spalte erzeugen

In [8]:
import hashlib

def hash(row):
    combined = (
        f"{row['period']} {row['session']} {row["person_id"]} {row["line1"]} {row["line2"]} {row["line3"]}"
    )
    return hashlib.sha256(combined.encode()).hexdigest()

haikus["id"] = haikus.apply(hash, axis=1)
haikus.head()

,period,session,date,person_id,line1,line2,line3,context_before,context_after,period_roman,person_name,parties,image_url,id
0,25,59,2015-01-21T00:00:00,51561,Es wird jedenfalls,gut sein für Österreich und,die Österreicher,Sehr geehrte Damen und Herren! Sehr geehrte Zu...,"Das meint offenbar auch die ÖVP, weil sie sag...",XXV,Dr. Johannes Hübner,[FPÖ],https://parlament.gv.at/dokument/bild/200695/2...,c568e09cf40de24e301f3832f16e25d18f0d8f7d41e654...
1,24,213,2013-07-03T00:00:00,30011,Diesbezüglich hat,die Zukunft im Burgenland,bereits begonnen,Herr Präsident! Herr Minister! Kolleginnen und...,Sämtliche Gemeinden rund um den Neusiedler Se...,XXIV,Erwin Preiner,[SPÖ],https://parlament.gv.at/dokument/bild/72616/72...,dcfcd447906c3b6777a8f93426552f647dab5f508abe1c...
2,24,51,2009-12-11T00:00:00,15465,Ich verstehe schon,dass Sie nicht hören wollen,wie die Zahlen sind,Sehr geehrte Frau Präsidentin! Hohes Haus! Mei...,Sie liegen aber auf dem Tisch. (Abg. Dr. Glaw...,XXIV,Dipl.-Ing. Josef Pröll,[ÖVP],https://parlament.gv.at/dokument/bild/21016/21...,885994806cb3807ede85572397a703b970cec43315e660...
3,25,55,2014-12-11T00:00:00,83139,Interessant ist,auch die Frage zur Zukunft,des ländlichen Raums,Sehr geehrter Herr Präsident! Sehr geehrter He...,"Wir wollen nämlich, dass auch die auf dem Lan...",XXV,Dr. Kathrin Nachbaur,"[STRONACH, ÖVP]",https://parlament.gv.at/dokument/bild/64016/64...,242a0f5afb8a5747368437d486c800793a1ede7755debf...
4,27,171,2022-09-21T00:00:00,78586,Warum müssen wir,das schon wieder so machen,wie bei der Cofag,Der Beginn dieser kurzen Debatte ist ein Sinnb...,Warum müssen wir schon wieder eine Blackbox m...,XXVII,"Christian Hafenecker, MA",[FPÖ],https://parlament.gv.at/dokument/bild/201431/2...,02659ce12c1f762583896d04e0e00b802d45e6e6a22a73...


haiku mit gleicher id (gleiches haiku in gleicher sitzung von gleicher person) auf ältestes reduzieren

In [9]:
haikus["context_before_length"] = haikus["context_before"].str.len()
haikus = haikus.sort_values(by=["date", "context_before_length"])
haikus = haikus.drop_duplicates(subset=["id"], keep="first")
haikus = haikus.drop(columns=["context_before_length"])
haikus.head()

,period,session,date,person_id,line1,line2,line3,context_before,context_after,period_roman,person_name,parties,image_url,id
2,24,51,2009-12-11T00:00:00,15465,Ich verstehe schon,dass Sie nicht hören wollen,wie die Zahlen sind,Sehr geehrte Frau Präsidentin! Hohes Haus! Mei...,Sie liegen aber auf dem Tisch. (Abg. Dr. Glaw...,XXIV,Dipl.-Ing. Josef Pröll,[ÖVP],https://parlament.gv.at/dokument/bild/21016/21...,885994806cb3807ede85572397a703b970cec43315e660...
1,24,213,2013-07-03T00:00:00,30011,Diesbezüglich hat,die Zukunft im Burgenland,bereits begonnen,Herr Präsident! Herr Minister! Kolleginnen und...,Sämtliche Gemeinden rund um den Neusiedler Se...,XXIV,Erwin Preiner,[SPÖ],https://parlament.gv.at/dokument/bild/72616/72...,dcfcd447906c3b6777a8f93426552f647dab5f508abe1c...
5,24,216,2013-07-05T00:00:00,178,Diese Frage ist,von den beiden Vorrednern,nicht gestellt worden,"Herr Kollege Hübner, ich weiß jetzt nicht, was...",(Abg. Strache: Hören Sie einmal zu!) Daher wä...,XXIV,Dr. Josef Cap,[SPÖ],https://parlament.gv.at/dokument/bild/34888/34...,ee5a9158af85eadabc71abd07bdcf9ffc68cc830fc1c56...
6,25,30,2014-06-12T00:00:00,35520,Wir allerdings sind,mehr und mehr draufgekommen,wo es überall stinkt,Herr Präsident! Meine sehr geehrten Damen und ...,Da habe ich vorher den Bereich der Pensionska...,XXV,Herbert Kickl,[FPÖ],https://parlament.gv.at/dokument/bild/201134/2...,e929ce4e82dfd616dc8d4bdd84827fdc0c423b9e5ab244...
7,25,38,2014-07-10T00:00:00,2822,Aus bekanntem Grund,mache ich das heute in,ihrer Vertretung,Meine sehr geehrten Damen und Herren! Geschätz...,"Ich tue das zunächst aber nicht, ohne der Fra...",XXV,Karlheinz Kopf,[ÖVP],https://parlament.gv.at/dokument/bild/200696/2...,ee472d5479f614a0aeb8b4e2e2d28ff67a95cfdf4bd259...


exportieren

In [10]:
haikus.to_json("web/haikus.json", orient="records", force_ascii=False, indent=2)